In [1]:
!python -m pip install --upgrade matrice

In [2]:
import yaml
import os

class Vars:
    _data = {}
    _path = None

    @classmethod
    def initialize(cls, path):
        cls._path = path
        cls._data = cls.load()

    @classmethod
    def load(cls):
        if cls._path and os.path.exists(cls._path):
            with open(cls._path, 'r') as f:
                return yaml.safe_load(f)
        return {}

    @classmethod
    def get(cls, key):
        if key not in cls._data:
            raise KeyError(f"Variable {key} not found")
        return cls._data[key]

    @classmethod
    def set(cls, key, value):
        cls._data[key] = value
        with open(cls._path, 'w') as f:
            yaml.safe_dump(cls._data, f, default_flow_style=False)


In [3]:
Vars.initialize('vars.yaml')

In [4]:
Vars.set("ENV", "dev")
Vars.set("MATRICE_ACCESS_KEY_ID", "IDAZ************MQO6")
Vars.set("MATRICE_SECRET_ACCESS_KEY", "52VR************ZY5N8")
Vars.set("MATRICE_ACCOUNT_NUMBER", "227684************7135")

In [6]:
import os
os.environ['ENV'] = Vars.get('ENV')
access_key = Vars.get('MATRICE_ACCESS_KEY_ID')
secret_key = Vars.get('MATRICE_SECRET_ACCESS_KEY')
account_number = Vars.get('MATRICE_ACCOUNT_NUMBER')

In [7]:
from matrice.session import Session
from matrice.projects import Projects
from matrice.dataset import Dataset
from matrice.models import Model
from matrice.deployment import Deployment
from matrice.exported_model import ExportedModel
import sys
import os

In [8]:
session_instance = Session.create_session(account_number=account_number, access_key=access_key, secret_key=secret_key)
print("A Session has been initialized:", session_instance)

A Session has been initialized: <matrice.session.Session object at 0x00000227901FB6D0>


In [12]:
projects_list, msg = session_instance.list_projects(project_type="classification")

# Check if there is a message or error
if msg:
    print(f"Message: {msg}")

# Print the projects in a formatted manner
for project_name, project_instance in projects_list.items():
    print(f"Project Name: {project_name} | Project ID: {project_instance.project_id}")

Message: Projects fetched successfully
Project Name: Shoplifting-Retail-Detection | Project ID: 6728cc001478c8033c87753b
Project Name: Crop-Weed-Detection | Project ID: 6728bd804e370c5e10793517
Project Name: Drone-Windmill-Maintainence | Project ID: 672251b73f122d1d31ac85df
Project Name: Test-Project-1 | Project ID: 67221278e444a8664263f806
Project Name: test-det | Project ID: 671f7af124f52600d8b27df2
Project Name: Drone Detection Use Case 1 | Project ID: 66c6fa777668fd51f7b98c74
Project Name: Crop Detection Use Case Final-4 | Project ID: 66c5cffd2dca8efe8f2562ec
Project Name: instance_segmentation | Project ID: 66a08a0ea6b455cdb12a5d59
Project Name: Demo1 | Project ID: 668b2bac4f0265d79523f23d
Project Name: kaggle_demo | Project ID: 66416ab972f6cd7a92435408


In [13]:
project_name = "Tumor-Classification-Project"
for name, project_instance in projects_list.items():
    if name == project_name:
        projects_instance = Projects(session_instance, project_name=project_name)
        session_instance.update(projects_instance.project_id)
        print(f"Project '{project_name}' already exists. Initialized Projects instance.")
        project_exists = True
        break
    else:
        project_exists = False

if project_exists == False:
    print(f"Creating a new project: {project_name}")

Creating a new project: Tumor-Classification-Project


In [14]:
import pprint

name = "Tumor-Classification-Project"

# Function to create a project
def create_project():
    project_name = name
    input_type = "image"
    output_type = "classification"
    
    enabled_platforms = {
        "matrice": True,
        "android": False,
        "ios": False,
        "tpu": False,
        "intelCPU": False,
        "gcloudGPU": False
    }

    resp, error = session_instance._create_project(project_name, input_type, output_type)
    if error:
        print(f"Error: {error}")
        return None, None
    else:
        print(f"Project created with ID: {resp['_id']}")
        return resp['_id'], resp['name']

# Check if project_id and project_name exist
project_id = None
project_name = None

if not project_exists:
    project_id, project_name = create_project()
else:
    project_id = projects_instance.project_id
    project_name = projects_instance.project_name
    print(f"Project already exists with ID: {project_id} and Name: {project_name}")


print("----------------------------")
print(f"Project ID: {project_id}")
print(f"Project Name: {project_name}")
print("----------------------------")


Project created with ID: 6728cfef1478c8033c877557
----------------------------
Project ID: 6728cfef1478c8033c877557
Project Name: Tumor-Classification-Project
----------------------------


In [15]:
Vars.set("project_name",project_name)
Vars.set("project_id", project_id)

In [16]:
session_instance.update(project_id)
project_instance = Projects(session_instance, project_name=project_name)
print("A Project class has been initialized",project_instance)

A Project class has been initialized <matrice.projects.Projects object at 0x00000227ECD38410>


In [17]:
dataset, dataset_action = project_instance.import_local_dataset(
    dataset_name='Tumor-Dataset',
    file_path = r"C:\Users\pathi\OneDrive\Desktop\matriceai\matrice_usecases_local\datasets\tumour_dataset_1.zip",
    dataset_type = "classification",
    dataset_description = "Dataset of Tumour Images",
    version_description = "1st version",
    input_type = "image"
)

In [18]:
dataset_id = dataset.dataset_id
action_id = dataset_action.action_id

print(f"Dataset ID: {dataset_id}")
print(f"Action ID: {action_id}")

Dataset ID: 6728d0fa0623cf13c1a4574d
Action ID: 6728d0fb1478c8033c877559


In [19]:
Vars.set("dataset_id", dataset.dataset_id)
Vars.set("dataset_action_id", dataset_action.action_id)

In [20]:
import time
import pprint

action_id = dataset_action.action_id
def check_dataset_status():
    while True:
        print(" ")
        print("-------Status------")
        D = Dataset(session_instance, dataset_id=dataset_id)
        status = D.version_status
        print("Status of dataset:", status)
        print("-------------------")
        if status == 'processed':
            print("---------Preprocesing Complete---------")
            print("Dataset processed, proceed with experiment creation.")
            print("---------------------------------------")
            break
        
        time.sleep(90)  # Wait for 90 seconds before checking again

# Run the function to check dataset status
check_dataset_status()


 
-------Status------
Status of dataset: queued
-------------------
 
-------Status------
Status of dataset: queued
-------------------
 
-------Status------
Status of dataset: processing
-------------------
 
-------Status------
Status of dataset: processing
-------------------
 
-------Status------
Status of dataset: processing
-------------------
 
-------Status------
Status of dataset: processing
-------------------
 
-------Status------
Status of dataset: processing
-------------------
 
-------Status------
Status of dataset: processing
-------------------
 
-------Status------
Status of dataset: processing
-------------------
 
-------Status------
Status of dataset: processing
-------------------
 
-------Status------
Status of dataset: processing
-------------------
 
-------Status------
Status of dataset: processing
-------------------
 
-------Status------
Status of dataset: processed
-------------------
---------Preprocesing Complete---------
Dataset processed, proceed with e

In [21]:
def print_dataset_info(dataset):
   
    dataset_id = dataset.dataset_details['_id']
    dataset_name = dataset.dataset_details['name']
    version_status = dataset.dataset_details.get("stats", [{}])[0].get("versionStatus")
    latest_version = dataset.dataset_details['latestVersion']
    no_of_samples = sum(version['versionStats']['total'] for version in dataset.dataset_details.get('stats', []))
    no_of_classes = len(dataset.dataset_details.get('stats', [{}])[0].get('classStat', {}))
    no_of_versions = len(dataset.dataset_details.get('allVersions', []))
    last_updated_at = dataset.dataset_details.get('updatedAt')

    print(f"Dataset ID: {dataset_id}")
    print(f"Dataset Name: {dataset_name}")
    print(f"Version Status: {version_status}")
    print(f"Latest Version: {latest_version}")
    print(f"Number of Samples: {no_of_samples}")
    print(f"Number of Classes: {no_of_classes}")
    print(f"Number of Versions: {no_of_versions}")
    print(f"Last Updated At: {last_updated_at}")

dataset.refresh()   
print_dataset_info(dataset)

Dataset ID: 6728d0fa0623cf13c1a4574d
Dataset Name: Tumor-Dataset
Version Status: processed
Latest Version: v1.0
Number of Samples: 253
Number of Classes: 2
Number of Versions: 1
Last Updated At: 2024-11-04T14:10:36.904Z


In [22]:
dataset_summary = dataset.summary

In [23]:
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio


# Convert the histogram data to a DataFrame
df = pd.DataFrame(dataset_summary['histogram'])

# Bar chart for counts by category
fig1 = go.Figure(data=[go.Bar(x=df['label'], y=df['count'])])
fig1.update_layout(
    title='Count of Items by Category',
    xaxis_title='Category',
    yaxis_title='Count'
)

# Pie chart for distribution of data items
fig2 = go.Figure(data=[go.Pie(
    labels=['Test', 'Train', 'Unassigned', 'Validation'],
    values=[dataset_summary['testDataItemCount'], dataset_summary['trainDataItemCount'], dataset_summary['unassignedDataItemCount'], dataset_summary['valDataItemCount']],
    hole=0.3
)])
fig2.update_layout(
    title='Distribution of Data Items'
)

# Display the figures
fig1.show()
fig2.show()


In [24]:
import os
import json
import pprint
import uuid

# Generate a unique experiment name
unique_experiment_name = "Tumor-Classification-Experiment"

# Function to create an experiment
def create_experiment():
    name = unique_experiment_name
    target_run_time = ["PyTorch"]
    primary_metric = "acc@1"
    matrice_compute = True
    dataset_version = 'v1.0'

    experiment_instance = project_instance.create_experiment(
        name,
        dataset_id,
        target_run_time[0],
        dataset_version,
        primary_metric,
        matrice_compute
    )

    print(experiment_instance)
    experiment_id = experiment_instance.experiment_id
    experiment_name = experiment_instance.experiment_name

    print(f"Experiment ID: {experiment_id}")
    print(f"Experiment Name: {experiment_name}")

    return experiment_instance, experiment_id, experiment_name

# Create a new experiment
experiment_instance , experiment_id, experiment_name = create_experiment()

print("----------------------------")
print(f"Experiment ID: {experiment_id}")
print(f"Experiment Name: {experiment_name}")
print(f"Experiment instance has been created: {experiment_instance}")
print("----------------------------")

[{'_id': '6728d0fa0623cf13c1a4574d', 'name': 'Tumor-Dataset', '_idProject': '6728cfef1478c8033c877557', 'status': '', 'createdAt': '2024-11-04T13:49:46.992Z', 'updatedAt': '2024-11-04T14:10:36.904Z', 'stats': [{'version': 'v1.0', 'versionStats': {'test': 26, 'train': 176, 'val': 51, 'unassigned': 0, 'total': 253}, 'versionDescription': '1st version', 'classStat': {'no_tumour': {'total': 98, 'train': 68, 'test': 10, 'val': 20, 'unassigned': 0}, 'tumour': {'total': 155, 'train': 108, 'test': 16, 'val': 31, 'unassigned': 0}}, 'versionStatus': 'processed', 'isFrozen': False, 'splitwiseBalance': True, 'classwiseBalance': False}], 'allVersions': ['v1.0'], 'processedVersions': ['v1.0'], 'latestVersion': 'v1.0', 'disabledVersions': None}]
Experiment ID: 66b35c3da8a2b1aa0504e1f1
Experiment Name: Tumor-Classification-Experiment
----------------------------
Experiment ID: 66b35c3da8a2b1aa0504e1f1
Experiment Name: Tumor-Classification-Experiment
Experiment instance has been created: <matrice.exper

In [25]:
Vars.set("experiment_id", experiment_instance.experiment_id)
Vars.set("experiment_name", experiment_instance.experiment_name)

In [26]:
from matrice.model_store import _get_all_model_families 

resp , error , mesage = _get_all_model_families( session_instance , project_id , project_type="classification" )
print("----List of available Detection models on platform----")
# Iterate through each model family in the response
for model_family in resp:
    # Extract _id and modelFamily (model family name)
    model_family_id = model_family.get('_id')
    name = model_family.get('modelFamily')
    
    # Print in formatted manner
    print(f"ID: {model_family_id}, Model Family Name: {name}")

----List of available Detection models on platform----
ID: 669529779bbb09931616d41d, Model Family Name: AlexNet
ID: 669529999bbb09931616d424, Model Family Name: CLIP
ID: 669529cc9bbb09931616d440, Model Family Name: DenseNet
ID: 669529f69bbb09931616d459, Model Family Name: EfficientNet
ID: 66952a359bbb09931616d48a, Model Family Name: EfficientNet V2
ID: 66952a939bbb09931616d4b3, Model Family Name: GoogleNet
ID: 66952ab19bbb09931616d4ba, Model Family Name: Inception V3
ID: 66952ad59bbb09931616d4c1, Model Family Name: MobileNet
ID: 66952b5e9bbb09931616d518, Model Family Name: ResNet
ID: 66952bbd9bbb09931616d54b, Model Family Name: ShuffleNet
ID: 66952be79bbb09931616d564, Model Family Name: SqueezeNet
ID: 66952c279bbb09931616d57e, Model Family Name: VGG
ID: 66952c5c9bbb09931616d5a9, Model Family Name: ViT
ID: 66952c889bbb09931616d5bc, Model Family Name: WideResNet
ID: 66c5a071a709d56d56038e47, Model Family Name: igwdjewq
ID: 66e1d60ff085f58fb95f3cab, Model Family Name: SwinTransformers
ID:

In [27]:
from matrice.model_store import ModelFamily

model_family_id = "66952b5e9bbb09931616d518"
# Initialize the ModelFamily instance after choosing a Model Family
model_family_instance = ModelFamily(session_instance, model_family_id=model_family_id) # ResNet
print("A ModelFamily instance has been initialized",model_family_instance)

A ModelFamily instance has been initialized <matrice.model_store.ModelFamily object at 0x0000022790B60FD0>


In [28]:
# Fetch model architectures
import pprint
arch_resp, error, message = model_family_instance.get_model_archs()
if error:
    print(f"Error: {error}")
else:
    pprint.pprint(arch_resp)
    # Check if resp is a list or a dictionary
    if isinstance(arch_resp, list):
        # Iterate through each model info in the list
        for model_info in arch_resp:
            model_key = model_info["model_key"]
            model_arch_instance = model_info["model_arch_instance"]
            
            # Extract model information
            model_info_id = model_arch_instance.model_info_id
            name = model_arch_instance.model_name
            key = model_arch_instance.model_key
            params = model_arch_instance.params_millions

            # Print in formatted manner
            print(f"ID: {model_info_id} |  Model Name: {name} | Model Key: {key} | Params in Millions: {params}")
            
    elif isinstance(arch_resp, dict):
        # Iterate through each model key in the dictionary
        for model_key, model_arch_instance in arch_resp.items():
            # Extract model information
            model_info_id = model_arch_instance.model_info_id
            name = model_arch_instance.model_name
            key = model_arch_instance.model_key
            params = model_arch_instance.params_millions

            # Print in formatted manner
            print(f"ID: {model_info_id} | Model Name: {name} | Model Key: {key} | Params in Millions: {params}")

{'resnet101': <matrice.model_store.ModelArch object at 0x000002278EB4E5D0>,
 'resnet152': <matrice.model_store.ModelArch object at 0x0000022790F76550>,
 'resnet18': <matrice.model_store.ModelArch object at 0x0000022790B6BF10>,
 'resnet34': <matrice.model_store.ModelArch object at 0x000002279020DB10>,
 'resnet50': <matrice.model_store.ModelArch object at 0x00000227904E6E10>}
ID: 66952b629bbb09931616d519 | Model Name: ResNet-101 | Model Key: resnet101 | Params in Millions: 42.8
ID: 66952b669bbb09931616d51a | Model Name: ResNet-152 | Model Key: resnet152 | Params in Millions: 58.5
ID: 66952b6a9bbb09931616d51b | Model Name: ResNet-18 | Model Key: resnet18 | Params in Millions: 11.4
ID: 66952b6e9bbb09931616d51c | Model Name: ResNet-34 | Model Key: resnet34 | Params in Millions: 21.5
ID: 66952b719bbb09931616d51d | Model Name: ResNet-50 | Model Key: resnet50 | Params in Millions: 23.9


In [29]:
from matrice.model_store import ModelArch
train_arch = 'resnet50' 
print("Chosen Training Architecture :" ,train_arch)
model_arch_instance = arch_resp.get(train_arch)
print("Model Architecture instance initialized for chosen training architecture :", model_arch_instance)

Chosen Training Architecture : resnet50
Model Architecture instance initialized for chosen training architecture : <matrice.model_store.ModelArch object at 0x00000227904E6E10>


In [30]:
import pprint
train_instance , train_config = model_arch_instance.get_train_config(experiment_id=Vars.get('experiment_id'))
print("-----Default Train Config------")
pprint.pprint(train_config)
print("--------------------------------")
print("-------Training instance initialized--------")
print(train_instance)

-----Default Train Config------
{'action_config': {'batch_size': {'allowedValues': [1, 2, 4, 8, 16, 32, 64],
                                  'anyValue': True,
                                  'defaultValue': 4,
                                  'hyperparameter': True,
                                  'keyName': 'batch_size',
                                  'maxValue': 1024,
                                  'minValue': 1,
                                  'selectedValues': [4],
                                  'valueType': 'int32'},
                   'epochs': {'allowedValues': [10,
                                                20,
                                                30,
                                                40,
                                                50,
                                                60,
                                                70,
                                                80,
                                      

In [31]:
model_config = train_config['model_config']

# Modify the model_config as needed
# modifications:
model_config['batch'] = [8]
model_config['epochs'] = [50]



# Repass the modified model_config into the train_config
train_config['model_config'] = model_config

# Print the updated train_config
print("-----Updated Model Config------")
pprint.pprint(train_config['model_config'])
print("--------------------------------")

# Print the updated train_config
print("-----Updated Train Config------")
pprint.pprint(train_config)
print("--------------------------------")

-----Updated Model Config------
{'batch': [8],
 'batch_size': [4],
 'epochs': [50],
 'learning_rate': [0.001],
 'lr_gamma': [0.1],
 'lr_min': [1e-05],
 'lr_scheduler': ['StepLR'],
 'lr_step_size': [10],
 'min_delta': [0.0001],
 'momentum': [0.95],
 'optimizer': ['AdamW'],
 'patience': [5],
 'weight_decay': [0.0001]}
--------------------------------
-----Updated Train Config------
{'action_config': {'batch_size': {'allowedValues': [1, 2, 4, 8, 16, 32, 64],
                                  'anyValue': True,
                                  'defaultValue': 4,
                                  'hyperparameter': True,
                                  'keyName': 'batch_size',
                                  'maxValue': 1024,
                                  'minValue': 1,
                                  'selectedValues': [4],
                                  'valueType': 'int32'},
                   'epochs': {'allowedValues': [10,
                                                20,

In [32]:
train_resp , msg , err = experiment_instance.add_models_for_training(train_instance,train_config)
print("------------Model added for training----------")
pprint.pprint(train_resp)

------------Model added for training----------
[{'_id': '6728d756388e36b4e54b5acf',
  '_idActionStatus': '6728d7561478c8033c87759a',
  '_idDataset': '66b34d94ae5a8a699aec688c',
  '_idExperiment': '66b35c3da8a2b1aa0504e1f1',
  '_idModelFamily': '000000000000000000000000',
  '_idModelInfo': '66952b719bbb09931616d51d',
  '_idProject': '6728cfef1478c8033c877557',
  '_idUser': '667996abc47fb97dea838b35',
  'actionConfig': {'batch_size': {'allowedValues': [1, 2, 4, 8, 16, 32, 64],
                                  'anyValue': True,
                                  'defaultValue': 4,
                                  'hyperparameter': True,
                                  'keyName': 'batch_size',
                                  'maxValue': 1024,
                                  'minValue': 1,
                                  'selectedValues': [4],
                                  'valueType': 'int32'},
                   'epochs': {'allowedValues': [10,
                               

In [ ]:
import time
from matrice.action import Action

# Define the action ID and the status you want to check
action_id = train_resp[0]['_idActionStatus']
success_status = 'SUCCESS'

# Function to check the status of the action
def check_action_status(S, action_id):
    action_instance = Action(S, action_id)
    print("---------------------")
    print(f"Action ID: {action_id}")
    print(f"Current Status: {action_instance.status}")
    print(f"Step Code: {action_instance.step_code}")
    print(f"Action service is : {action_instance.service_name}")
    print("---------------------")
    return action_instance.status

# Loop to check status every 2 minutes until it is 'success'
while True:
    status = check_action_status(session_instance, action_id)
    if status == success_status:
        print("Action status is 'success'. Model is successfully trained.")
        break
    else:
        print("Model is training. Waiting 10 minutes before checking again.")
        time.sleep(400)  # Wait for 10 minutes (600 seconds)


---------------------
Action ID: 6728c5601478c8033c877525
Current Status: OK
Step Code: MDL_TRN_STRT
Action service is : be-model
---------------------
Model is training. Waiting 10 minutes before checking again.


In [ ]:
model_id = train_resp[0]['_id']
print(f"Model ID from response data: {model_id}")
    

# Initialize the Model class with the model_id
model_instance = Model(session_instance, model_id)
print("A Model instance has been initialized : ", model_instance)

In [ ]:
# Print the test score, validation score, and best epoch
print(f"Test Score: {model_instance.test_score}")
print(f"Validation Score: {model_instance.val_score}")
print(f"Best Epoch: {model_instance.best_epoch}")

In [ ]:
Vars.set("model_train_id", model_instance.model_id)
Vars.set("model_train_name", model_instance.model_train_name)

In [ ]:
model_instance.plot_epochs_losses()

In [ ]:
model_instance.plot_epochs_metrics()

In [ ]:
model_instance.plot_eval_results()

In [ ]:
import os
import random
import requests
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# Define the folder path containing the images
folder_path = r"C:\Users\pathi\OneDrive\Desktop\matriceai\matrice_usecases_local\datasets\test"


# Get a list of all image files in the folder
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Randomly select 10 images from the folder
random_images = random.sample(image_files, 2)

print(random_images)
# Loop through the selected images and make predictions
for image_file in random_images:
    # Get the prediction results from the model
    result, error, _ = model_instance.get_prediction(image_file)
    print(result)
    if error:
        print(f"Error: {error}")
        continue
    
    # Extract the predicted class label and confidence
    predicted_class = result["category"]
    confidence = result["confidence"]
    
    # Open the image
    image = Image.open(image_file)
    
    # Create a matplotlib figure and axis
    plt.figure(figsize=(10, 10))
    fig, ax = plt.subplots(1)
    
    # Display the image
    ax.imshow(image)
    
    # Add the predicted class label and confidence to the image
    plt.text(10, 10, f"{predicted_class} ({confidence:.2f})", bbox=dict(facecolor='yellow', alpha=0.5))
    
    # Display the image with the annotation
    plt.show()


In [ ]:
model_instance.download_model(file_name="model.pt")

In [ ]:
export_config = train_instance.get_default_export_config("ONNX") # Get the default export config for export format user wants (here ONNX)
print("Export Config for ONNX:")
pprint.pprint(export_config)

In [ ]:
exported_instance , action_instance = project_instance.create_model_export(model_id,"ONNX",export_config)

print("Model Export class has been initialized :", exported_instance )   
print("----------------------------")
print(f"Export ID: {exported_instance.model_export_id}")
print(f"Export Name: {exported_instance.model_export_name}")
print(f"Action ID: {action_instance.action_id}")
print(f"Action Status: {action_instance.status}")
print("----------------------------")

In [ ]:
Vars.set("model_export_id", exported_instance.model_export_id)
Vars.set("model_export_name", exported_instance.model_export_name)

In [ ]:
import time
from matrice.action import Action

# Define the action ID and the status you want to check
action_id = action_instance.action_id
success_status = 'SUCCESS'

# Function to check the status of the action
def check_action_status(S, action_id):
    A = Action(S, action_id)
    print("---------------------")
    print(f"Action ID: {action_id}")
    print(f"Current Status: {A.status}")
    print(f"Step Code: {A.step_code}")
    print(f"Action service is : {A.service_name}")
    print("---------------------")
    return A.status

# Loop to check status every 1.5 minutes until it is 'success'
while True:
    status = check_action_status(session_instance, action_id)
    if status == success_status:
        print("Action status is 'success'. Model is successfully exported.")
        break
    else:
        print("Model is exporting. Waiting 2 minutes before checking again.")
        time.sleep(90)  # Wait for 1.5 minutes (90 seconds)


In [ ]:
exported_instance.download_model(file_name="model.onnx")

In [ ]:
deployment , deployment_action = project_instance.create_deployment(
    model_id = Vars.get('model_export_id'),  # It can also be model_train_id depending on the model you want to deploy
    deployment_name = "deployment_name",
    shutdown_threshold=20
)

In [ ]:
key_resp = deployment.create_auth_key()
auth_key = key_resp['key']

In [ ]:
Vars.set("deployment_id", deployment.deployment_id)
Vars.set("deployment_name", deployment.deployment_name)
Vars.set("auth_key", auth_key)

In [ ]:
import os
import random
import requests
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# Define the folder path containing the images
folder_path = r"<path_to_test_folder>"


# Get a list of all image files in the folder
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Randomly select 10 images from the folder
random_images = random.sample(image_files, 10)


# Loop through the selected images and send the prediction request
for image_file in random_images:
    
    result, error, message = deployment.get_prediction(auth_key,image_file)

    if error:
        print(f"Error: {error}")
        continue

    # Extract prediction data from the result
    predictions = result

    # Open the image
    image = Image.open(image_file)
    plt.figure(figsize=(10, 10))
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    # Draw bounding boxes and labels
    for prediction in predictions:
        category = prediction["category"]
        confidence = prediction["confidence"]
        bbox = prediction["bounding_box"]

        # Extract the bounding box coordinates
        xmin, ymin, xmax, ymax = bbox["xmin"], bbox["ymin"], bbox["xmax"], bbox["ymax"]
        width, height = xmax - xmin, ymax - ymin

        # Create a Rectangle patch
        rect = plt.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(xmin, ymin, f"{category} ({confidence:.2f})", bbox=dict(facecolor='yellow', alpha=0.5))

    plt.show()
